# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №6 </center>

На этой неделе вам предстоит написать Дерево рещений своими руками и оптимизировать его. 

Импортируем библиотеки которые нам нужны

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score as f1
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Импортируем нужные данные, в этот раз мы будем пытаться предсказать ценовой диапазон цены на телефон. Данные уже обработаны

In [2]:
df = pd.read_csv('/Users/ivan/DS_educate/Courses_ipynb/REU/data/mobile_price_prediction.csv')
print(df.info())
print("\n PRICE_RANGE UNIQUE VALUES: ", df["price_range"].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

Как мы видим у нас есть 4 класса, разделите данные на train и test. test_size = 0.25 и random_state = 1

In [3]:
features = df.columns[:-1].to_list()
target = df.columns[-1] # price_range
X = df[features]
y = df[target]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# **Задание 1. Функции для оптимизации (2 балла)**

Для начала реализуем критерий Gini по которому мы будем оптимизировать наше дерево. Чем данный критерий меньше, тем лучше новое разделение разделяет данные. На вход нашей функции будет подаваться массив с значениями y. Например [0, 1, 3, 2, 1, 0, 0,0 ]

$$ Gini = 1 - {\sum_j}p_j^2 $$

![image-7.png](https://wikimedia.org/api/rest_v1/media/math/render/svg/1fe8179fbf8ca4addf0e8f5f404fdbf153a04214)

In [5]:
def gini(array):
    n = len(array)
    freq = {}
    for item in array:
        if item in freq:
            freq[item] += 1
        else:
            freq[item] = 1
    p = [float(freq[x]) / n for x in freq]
    return 1 - sum([x ** 2 for x in p])


In [6]:
#define NumPy array of income values
incomes = [0, 1, 1, 1, 0]

#calculate Gini coefficient for array of incomes
gini(incomes)

0.48

 Проверьте правильно ли работает ваша функция. Если вы запустили код ниже и он отработал без ошибок, то значит все хорошо

In [7]:
test_cases = [
    ([0, 0, 0, 0, 0], 0.0),
    ([0, 1, 1, 1, 0], 0.48),
    ([0, 1, 2, 3, 5, 5, 2], 0.7755102040816326)
]

# Test the gini function against the test cases
for x, expected in test_cases:
    result = gini(x)
    if result == expected:
        print(f"Test case PASSED: gini({x}) = {result}")
    else:
        raise ValueError(f"Test case FAILED: gini({x}) = {result}, expected {expected}")


Test case PASSED: gini([0, 0, 0, 0, 0]) = 0.0
Test case PASSED: gini([0, 1, 1, 1, 0]) = 0.48
Test case PASSED: gini([0, 1, 2, 3, 5, 5, 2]) = 0.7755102040816326


# **Задание 2. Дерево решений (13 баллов)**

Теперь напишем сам алгоритм, в нем обязательно нужно настроить возможность ограничения дерева по глубине. Алгоритм обучения вы можете написать в удобном вам виде, главное чтобы он работал

In [8]:
class MyDecisionTree:
    def __init__(self, max_depth, min_samples_split):
        self.max_depth = max_depth   # задаем максимальную глубину
        self.min_samples_split = min_samples_split # миниальное число наблюдений для разбиения

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y)) # количество классов
        self.tree_ = self._plant_tree(X, y) # строим дерево 

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    # Функция которая поможет найти лучший признак для разбиения в наших данных
    def _best_split(self, X, y):

        num_of_y = y.size # Сразу проверим что у нас больше чем одно наблюдение
        if num_of_y <= 1:
            return None, None

        # Создадим list, где у нас будет указано количество каждого класса
        num_of_each_class = [np.sum(y == c) for c in range(self.n_classes_)]
        
        # Ваш код тут, вам нужно:
        # 1) Вернуть индекс колонки где лучше всего разбить данные
        # 2) Найти в этой колоники лучшее значения для разбиения и вернуть их
        # Также учитывайте, что если после рaзбиения в одном из листьев у вас меньше
        # чем min_samples_split, вам не подходит такое разбиение
        best_idx = None
        best_thr = None
        best_gini = 1.0
        for idx in range(X.shape[1]):
          thresholds = np.unique(X[:, idx])
          for thr in thresholds:
              left_indices = X[:, idx] < thr
              right_indices = ~left_indices
              if np.sum(left_indices) < self.min_samples_split or np.sum(right_indices) < self.min_samples_split:
                  # Пропускаем разбиение, если один из листьев имеет меньше, чем min_samples_split
                  continue

        # Важный момент, при подсчете gini после разбиения, нужно подсчитать его как для
        # правого листа, так и для левого, а затем расччитать общий gini. 
        # gini = (i * gini_left + (m - i) * gini_right) / m 
        # i - количество наблюдений в левом листе, m - в правом
              gini_all = (np.sum(left_indices) * gini(y[left_indices]) + np.sum(right_indices) * gini(y[right_indices])) / num_of_y
              if gini_all < best_gini:
                  best_gini = gini_all
                  best_idx = idx
                  best_thr = thr
        return best_idx, best_thr

    def _plant_tree(self, X, y, depth=0):

        # Для начала мы предскажем класс, взяв максимальным тот, который чаще всего встречается в данных 
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)

        # Тут нам будет помогать другой класс Node, по-другому лист.
        # Мы будем использовать его для сохранения информации о разбиении
        node = Node(predicted_class=predicted_class)

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                
                # Ваш код тут. Создайте переменные со значениями X и y которые отправлись в правый лист.
                # создайте также переменные с значениями, которые отправились в левый лист
                # Другими словами просто разделите данные на две части по вашему best_split
                left_indices = X[:, idx] < thr
                right_indices = ~left_indices

                node.feature_index = idx
                node.threshold = thr
                node.left = self._plant_tree(X[left_indices], y[left_indices], depth + 1) # вызовите plant_tree и передайте туда ваши переменные в левой части, к глубине прибавьте +1
                node.right = self._plant_tree(X[right_indices], y[right_indices], depth + 1) # вызовите plant_tree и передайте туда ваши переменные в правой части, к глубине прибавьте +1

        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

class Node:
    def __init__(self, *, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


In [30]:
# Ваш код здесь
model=MyDecisionTree(max_depth=5, min_samples_split=2)
model.fit(X_train.values,y_train.values)
y_pred=model.predict(X_test.values)
f1_score = f1(y_test, y_pred, average='macro')
print("max_depth=5, min_samples_split=2 \n F1 Score:", round(f1_score, 3))

max_depth=5, min_samples_split=2 
 F1 Score: 0.824


In [43]:
model=MyDecisionTree(max_depth=30, min_samples_split=8)
model.fit(X_train.values,y_train.values)
y_pred=model.predict(X_test.values)
f1_score = f1(y_test, y_pred, average='macro')
print("max_depth=30, min_samples_split=8 \n F1 Score:", round(f1_score, 3))

max_depth=30, min_samples_split=8 
 F1 Score: 0.835


In [44]:
model=MyDecisionTree(max_depth=1, min_samples_split=1)
model.fit(X_train.values,y_train.values)
y_pred=model.predict(X_test.values)
f1_score = f1(y_test, y_pred, average='macro')
print("max_depth=1, min_samples_split=1 \n F1 Score:", round(f1_score, 3))

max_depth=1, min_samples_split=1 
 F1 Score: 0.323


In [47]:
model=MyDecisionTree(max_depth=2, min_samples_split=5)
model.fit(X_train.values,y_train.values)
y_pred=model.predict(X_test.values)
f1_score = f1(y_test, y_pred, average='macro')
print("max_depth=2, min_samples_split=5 \n F1 Score:", round(f1_score, 3))

max_depth=2, min_samples_split=5 
 F1 Score: 0.739


# **Задание 3. Деревья в Sklearn (5 баллов)**

Напишите поиск параметров(их должно быть минимум 4) с помощью GridSearchCV дл RandomForestClassifier. Он может долго перебирать параметры, это нормально. Оцените результат с помощью f1_score, укажите в нем average = 'marco'

In [ ]:
gs_model = RandomForestClassifier()
params = {
    'max_depth': [1, 5, 10],
    'n_estimators': [5, 20, 100],
}

gs = GridSearchCV(gs_model, params, cv=3, scoring='accuracy', verbose=3)
gs.fit(X_train, y_train)

print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

In [ ]:
preds_train = gs.predict(X_train)
preds_test = gs.predict(X_test)

In [ ]:
print("\nТочность на обучении f1:", f1(y_train, preds_train, average='macro'))
print("Точность на тесте f1:", f1(y_test, preds_test, average='macro'))

![pic1.png](https://memesmix.net/media/created/ivas8z.jpg)